In [1]:
import mlrun

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


Load a Project from exicting project YAML, you can clone you entire repository from git and use the project YAML in your repository using the url parameter

In [2]:
project = mlrun.load_project('./')

In [3]:
print(project.to_yaml())

kind: project
metadata:
  name: from-git
spec:
  functions:
  - name: nuclio
    kind: nuclio
    image: mlrun/mlrun
    handler: multi:multi_3
    with_repo: true
  - name: my_job
    kind: job
    image: mlrun/mlrun
    handler: my_job.func
    with_repo: true
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/load-project-example.git
  load_source_on_run: true
  desired_state: online



In [4]:
source = 'git://github.com/GiladShapira94/load-project-example.git#master'

In [5]:
project.set_source(source=source,pull_at_runtime=True)

Run a MLRun job if needed you can use the project.build_function for building a new image with specials requirments 

In [6]:
project.run_function(function='my_job',params={'num':3})

> 2022-07-24 13:34:43,823 [info] starting run my_job-func uid=842765f822fe4c5ca6deee92dd043563 DB=http://mlrun-api:8080
> 2022-07-24 13:34:44,040 [info] Job is running in the background, pod: my-job-func-r96kn
> 2022-07-24 13:34:48,121 [info] extracting source from git://github.com/GiladShapira94/load-project-example.git#master to /mlrun/code
24
> 2022-07-24 13:34:48,246 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
from-git,...dd043563,0,Jul 24 13:34:48,completed,my_job-func,v3io_user=giladkind=jobowner=giladmlrun/client_version=1.1.0-rc12host=my-job-func-r96kn,,num=3,num=24return=24,


> 2022-07-24 13:34:50,429 [info] run executed, status=completed


Changed pull_at_runtime to False for pull files on image creation

In [12]:
project.set_source(source=source)
print(project.to_yaml())

kind: project
metadata:
  name: from-git
spec:
  functions:
  - name: nuclio
    kind: nuclio
    image: mlrun/mlrun
    handler: multi:multi_3
    with_repo: true
  - name: my_job
    kind: job
    image: mlrun/mlrun
    handler: my_job.func
    with_repo: true
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/load-project-example.git#master
  load_source_on_run: false
  desired_state: online



Deploy Serving function

In [13]:
serving_func=project.deploy_function('serving')

> 2022-07-24 14:02:57,684 [info] Starting remote function deploy
2022-07-24 14:03:13  (info) Deploying function
2022-07-24 14:03:13  (info) Building
2022-07-24 14:03:13  (info) Staging files and preparing base images
2022-07-24 14:03:13  (info) Building processor image
2022-07-24 14:04:59  (info) Build complete
2022-07-24 14:05:07  (info) Function deploy complete
> 2022-07-24 14:05:07,901 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-from-git-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['from-git-serving-from-git.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/']}


In [14]:
serving_func.function.invoke('/',5)

> 2022-07-24 14:05:07,972 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-from-git-serving.default-tenant.svc.cluster.local:8080/'}


13

You can execute functions locally for test and debug, just turn on the local=True flag - > you must have the fucntion file in the local repository 

Deploy nuclio function

In [15]:
# project.spec.origin_url='git://github.com/GiladShapira94/load-project-example.git#refs/heads/master'
nuclio_func=project.deploy_function(function='nuclio')

> 2022-07-24 14:05:08,021 [info] Starting remote function deploy
2022-07-24 14:05:08  (info) Deploying function
2022-07-24 14:05:08  (info) Building
2022-07-24 14:05:08  (info) Staging files and preparing base images
2022-07-24 14:05:08  (info) Building processor image
2022-07-24 14:06:04  (info) Build complete
2022-07-24 14:06:12  (info) Function deploy complete
> 2022-07-24 14:06:12,748 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-from-git-nuclio.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['from-git-nuclio-from-git.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/']}


In [16]:
nuclio_func.function.invoke('/',{'int':4})

> 2022-07-24 14:06:12,877 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-from-git-nuclio.default-tenant.svc.cluster.local:8080/'}


{'int': 12}